In [1]:
import pandas as pd
import numpy as np
import spotipy 
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
cid ='96c23a48f808449f88bd8d8f44962e40' 
secret ='596d6b18cd104b79b9566e9776274c6d'

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False

# Getting all playlists under "Focus" and "Party" genres
focus = (sp.category_playlists(category_id="focus", country="US", limit=50, offset=0))
party = (sp.category_playlists(category_id="party", country="US", limit=50, offset=0))

def playlist_ids(X):
    playlist_uris = []
    for i,ids in enumerate(X["playlists"]['items']):
        playlist_uris.append(ids["uri"])
    return playlist_uris

party_uris = playlist_ids(party)
focus_uris = playlist_ids(focus)

In [3]:
def songs_database(playlist_uris):
    songs_df = pd.DataFrame()
    for uri in playlist_uris:
        playlist = sp.user_playlist("spotify",uri) 
        songs = playlist["tracks"]["items"] 
        ids = []
        song_names = []
        song_artist = []
        features = []
        
        for i in range(len(songs)): 
            ids.append(songs[i]["track"]["id"])
            song_names.append(songs[i]["track"]["name"])
            song_artist.append(songs[i]["track"]["artists"][0]['name'])
            f = sp.audio_features(songs[i]["track"]["id"])
            if(f != [None]):
                features.append(f)
        
        df1 = pd.DataFrame(features)
        df2 = pd.DataFrame()
        df2["track_title"] = song_names
        df2["track_artist"] = song_artist
        df2["playlist_name"] = playlist["name"]
        df3 = pd.concat([df2,df1], axis=1)
        
        if songs_df.empty:
            songs_df = df3
        else:
            songs_df = pd.concat([songs_df,df3],axis=0,ignore_index=True)
    
    return songs_df

In [5]:
party_songs = songs_database(party_uris)
party_songs

,track_title,track_artist,playlist_name,0
0,Temperature,Sean Paul,Throwback Jams,"{'danceability': 0.951, 'energy': 0.6, 'key': ..."
1,This Is How We Do It,Montell Jordan,Throwback Jams,"{'danceability': 0.799, 'energy': 0.623, 'key'..."
2,Can't Nobody Hold Me Down (feat. Mase),Diddy,Throwback Jams,"{'danceability': 0.928, 'energy': 0.771, 'key'..."
3,Back That Azz Up,Juvenile,Throwback Jams,"{'danceability': 0.874, 'energy': 0.714, 'key'..."
4,B.O.B. - Bombs Over Baghdad,OutKast,Throwback Jams,"{'danceability': 0.746, 'energy': 0.978, 'key'..."
...,...,...,...,...
2953,I'm A Rover,The Wild Rover,Irish Pub Songs,"{'danceability': 0.756, 'energy': 0.67, 'key':..."
2954,Paddle Me Own Canoe,Waxies Dargle,Irish Pub Songs,"{'danceability': 0.589, 'energy': 0.702, 'key'..."
2955,Kelly The Boy From Killane,The Fighting Men From Crossmaglen,Irish Pub Songs,"{'danceability': 0.662, 'energy': 0.489, 'key'..."
2956,The Rising Of The Moon,The High Kings,Irish Pub Songs,"{'danceability': 0.453, 'energy': 0.716, 'key'..."


In [4]:
party_songs.to_csv("party_songs.csv",index=False,header=True)

NameError: name 'party_songs' is not defined

In [ ]:
focus_songs = songs_database(focus_uris)
focus_songs

In [ ]:
focus_songs.to_csv("focus_songs.csv",index = False, header=True)

In [ ]:
focus_songs["time_signature"].value_counts()

In [ ]:
party_songs["time_signature"].value_counts()

In [ ]:
pcorr = party_songs[['danceability','energy', 'loudness', 'acousticness','instrumentalness', 'liveness']].corr()
pcorr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

party_c = plt.figure()
plt.figure(figsize = (8,8))
# mask = np.zeros_like(party_songs.corr())
# mask[np.triu_indices_from(mask)] = True
sns.heatmap(pcorr,cmap="RdBu_r", square=True, vmin=-1, vmax=1)
plt.title('Correlation matrix for Party songs')
plt.show()

party_c.savefig("party_corr.png", bbox_inches='tight', dpi=300)

In [ ]:
fcorr = focus_songs[['danceability','energy', 'loudness', 'acousticness','instrumentalness', 'liveness']].corr()
fcorr

In [ ]:
focus_c = plt.figure()
plt.figure(figsize = (8,8))
# mask2 = np.zeros_like(fcorr)
# mask2[np.triu_indices_from(mask2)] = True
sns.heatmap(fcorr,cmap="RdBu_r",square=True, vmin=-1, vmax=1)
plt.title('Correlation matrix for Focus songs')
plt.show()

focus_c.savefig("focus_corr.png", dpi=300)

In [ ]:
train_cols = ['danceability','energy', 'key', 'loudness', 'speechiness', 'acousticness',
              'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
target_cols = "genre_class"

In [ ]:
party_songs[target_cols] = 0
#party_songs
focus_songs[target_cols] = 1
#focus_songs

In [ ]:
complete_songs = pd.concat([party_songs,focus_songs],axis=0)
complete_songs = complete_songs.sample(frac=1).reset_index(drop=True)
#complete_songs

In [ ]:
features = complete_songs[train_cols]
target = complete_songs["genre_class"]
type(target)

In [ ]:
party_features_mean = party_songs[train_cols].mean()
focus_features_mean = focus_songs[train_cols].mean()
party_features_mean["duration_ms"] = party_features_mean["duration_ms"]/1000
focus_features_mean["duration_ms"] = focus_features_mean["duration_ms"]/1000
party_features_mean2 = (np.log(party_features_mean))
focus_features_mean2 = (np.log(focus_features_mean))

party_features_mean2 = party_features_mean2.drop(["loudness"])
focus_features_mean2 = focus_features_mean2.drop(["loudness"])

f = plt.figure()
#plt.style.use('fivethirtyeight')
party_features_mean2.plot.bar(color = 'r', align = 'center', label = 'Party', width = .25)
focus_features_mean2.plot.bar(color = 'b', align = 'edge', label = 'Focus', width = .25,
                         rot = 90, figsize = (8,8))

plt.title('Comparing Genre features: Party vs Focus', y = 1.1)
plt.ylim(-3,6)
plt.yticks(np.arange(0,5.1,.5))
plt.ylabel('Normalized Features')
plt.legend(loc="upper center")
plt.show()

f.savefig("bar_chart.png", bbox_inches='tight', dpi=300)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=1)
y_test = y_test.reset_index(drop=True)
y_test

In [ ]:
from sklearn.linear_model import LogisticRegression

logit_reg = LogisticRegression(solver='liblinear')
logit_reg.fit(X_train, y_train)
predictions = logit_reg.predict(X_test)
predictions = pd.Series(predictions)
predictions

In [ ]:
# False positives.
fp_filter = (predictions == 1) & (y_test == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (y_test == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (y_test == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (y_test == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
precision = tp/(tp+fp)

print("Recall:",tpr)
print("False Positive Rate:",fpr)
print("Precision:",precision)

In [ ]:
# Feature importance
importance = logit_reg.coef_[0]

# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# predict probabilities
class_probs = logit_reg.predict_proba(X_test)

# keep probabilities for the positive outcome only
class_probs = class_probs[:, 1]

# calculate scores
lr_auc = roc_auc_score(y_test, class_probs)

# summarize scores
print('Logistic: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
lr_fpr, lr_tpr, _ = roc_curve(y_test, class_probs)

# plot the roc curve for the model
roc_fig = plt.figure()
plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

roc_fig.savefig("ROC.png", bbox_inches='tight', dpi=600)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc

# predict class values
lr_precision, lr_recall, _ = precision_recall_curve(y_test, class_probs)
lr_f1, lr_auc = f1_score(y_test, predictions), auc(lr_recall, lr_precision)

# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

# plot the precision-recall curves
auc_fig = plt.figure()
plt.plot(lr_recall, lr_precision, marker='.', label='Logistic')

# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')

# show the legend
plt.legend()
# show the plot
plt.show()
auc_fig.savefig("auC.png", bbox_inches='tight', dpi=600)

In [ ]:
import pickle

# saving the model to disk
filename = 'final_logit_model.sav'
pickle.dump(logit_reg, open(filename, 'wb'))

In [ ]:
from sklearn.model_selection import cross_val_predict

logit_reg = LogisticRegression(solver='liblinear')
predictions = cross_val_predict(logit_reg, features, target, cv=5)
predictions = pd.Series(predictions)
predictions.value_counts()

In [ ]:
# False positives.
fp_filter = (predictions == 1) & (complete_songs["genre_class"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (complete_songs["genre_class"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (complete_songs["genre_class"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (complete_songs["genre_class"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
precision = tp/(tp+fp)

print("Recall:",tpr)
print("False Positive Rate:",fpr)
print("Precision:",precision)

In [ ]:
# predict probabilities
class_probs = cross_val_predict(logit_reg, features, target, cv=5,method="predict_proba")

# keep probabilities for the positive outcome only
class_probs = class_probs[:, 1]

# calculate scores
lr_auc = roc_auc_score(target, class_probs)

# summarize scores
print('Logistic: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
lr_fpr, lr_tpr, _ = roc_curve(target, class_probs)

# plot the roc curve for the model
plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
# predict class values
predictions = cross_val_predict(logit_reg, features, target, cv=5)
lr_precision, lr_recall, _ = precision_recall_curve(target, class_probs)
lr_f1, lr_auc = f1_score(target, predictions), auc(lr_recall, lr_precision)

# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

# plot the precision-recall curves
plt.plot(lr_recall, lr_precision, marker='.', label='Logistic')

# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')

# show the legend
plt.legend()
# show the plot
plt.show()